In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [115]:
spark = SparkSession.builder.appName('pyspark').getOrCreate()

FILENAME = '../cik.csv'
# regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,tik_ballots,
# bad_ballots,normal_ballots,missing_ballots,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky
schema = StructType([
    StructField('region', StringType(), False),
    StructField('tik', StringType(), False),
    StructField('uik', StringType(), False),
    StructField('registered_voters', IntegerType(), False),
    StructField('total_ballots', IntegerType(), False),
    StructField('early_voters', IntegerType(), False),
    StructField('tik_voters', IntegerType(), False),
    StructField('home_voters', IntegerType(), False),
    StructField('empty_ballots', IntegerType(), False),
    StructField('home_ballots', IntegerType(), False),
    StructField('tik_ballots', IntegerType(), False),
    StructField('bad_ballots', IntegerType(), False),
    StructField('normal_ballots', IntegerType(), False),
    StructField('missing_ballots', IntegerType(), False),
    StructField('extra_ballots', IntegerType(), False),
    StructField('baburin', IntegerType(), False),
    StructField('grudinin', IntegerType(), False),
    StructField('zhirinovsky', IntegerType(), False),
    StructField('putin', IntegerType(), False),
    StructField('sobchak', IntegerType(), False),
    StructField('suraykin', IntegerType(), False),
    StructField('titov', IntegerType(), False),
    StructField('yavlinsky', IntegerType(), False),
])

df = spark.read.csv(FILENAME, schema=schema)



Явка (%) по всем регионам, результат отсортирован по убыванию

In [116]:
counted_votes_sum_cols = ['early_voters_sum', 'tik_voters_sum', 'home_voters_sum']

grouped = df.groupBy(['region']) \
    .agg(F.sum('early_voters').alias('early_voters_sum'),
         F.sum('tik_voters').alias('tik_voters_sum'),
         F.sum('home_voters').alias('home_voters_sum'),
         F.sum('registered_voters').alias('registered_voters_sum'))\
    .dropna()

appearance = grouped.withColumn('appearance',
  (sum([F.col(col) for col in counted_votes_sum_cols]) / F.col('registered_voters_sum')))
      
appearance.sort('appearance', ascending=False).select(['region', 'appearance']).show()



+--------------------+------------------+
|              region|        appearance|
+--------------------+------------------+
|Территория за пре...|0.9806986984380843|
|     Республика Тыва|0.9366369316170003|
|Ямало-Ненецкий ав...|0.9190125747324195|
|Кабардино-Балкарс...|0.9180157863562131|
|Республика Северн...|0.8998810071492768|
| Республика Дагестан|0.8747795414462081|
|Карачаево-Черкесс...|0.8740544668203033|
| Кемеровская область|0.8322733516003962|
|Чукотский автоном...|0.8228138695924391|
|Республика Ингушетия|0.8195937165135314|
|    Брянская область| 0.797078003370434|
|   Тюменская область|0.7892544508025854|
|  Краснодарский край|0.7786760339047925|
|Республика Татарс...|0.7742157684137118|
|Чувашская Республ...|0.7622296238501433|
|Республика Башкор...|0.7544523455707033|
|  Республика Бурятия|0.7519978782127258|
|Республика Адыгея...|0.7431189118555477|
| Ставропольский край|0.7384811380095883|
|  Пензенская область|0.7374614571776765|
+--------------------+------------

Выбрать произвольного кандидата и найти тот избирательный участок,
на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [114]:
counted_votes_cols = ['early_voters', 'tik_voters', 'home_voters']
canditate = 'zhirinovsky'
voted = df.withColumn('total_voted', sum([F.col(col) for col in counted_votes_cols]))
canditate_votes = voted.filter(voted.total_voted > 300) \
    .select(
        df.region,
        df.tik,
        df.uik,
        (F.col(canditate)/df.normal_ballots).alias('percent'),
         F.col(canditate),
         F.col('total_voted')
).dropna() \
.sort(F.col('percent'), ascending=False) \
.show(1)

+--------------------+--------------------+---------+----------------+-----------+-----------+
|              region|                 tik|      uik|         percent|zhirinovsky|total_voted|
+--------------------+--------------------+---------+----------------+-----------+-----------+
|Новгородская область|Новгородcкая горо...|УИК №1194|0.39119804400978|        160|        422|
+--------------------+--------------------+---------+----------------+-----------+-----------+
only showing top 1 row



Регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна (%)

In [96]:
grouped = df.groupBy(['region', 'tik']) \
    .agg(F.sum('early_voters').alias('early_voters_sum'),
         F.sum('tik_voters').alias('tik_voters_sum'),
         F.sum('home_voters').alias('home_voters_sum'),
         F.sum('registered_voters').alias('registered_voters_sum'))\
    .dropna()

appearance = grouped.withColumn('appearance',
  (sum([F.col(col) for col in counted_votes_sum_cols]) / F.col('registered_voters_sum')))


appearance_diff = appearance.drop('early_voters_sum', 'tik_voters_sum', 'home_voters_sum', 'registered_voters_sum')

appearance_diff = appearance_diff.groupBy(['region']) \
    .agg(F.max('appearance').alias('max_ap'),
         F.min('appearance').alias('min_ap'))

appearance_diff = appearance_diff.withColumn('diff', appearance_diff.max_ap - appearance_diff.min_ap) \
.sort('diff', ascending=False)

appearance_diff.show(1)

+--------------------+----------------+-------------------+-------------------+
|              region|          max_ap|             min_ap|               diff|
+--------------------+----------------+-------------------+-------------------+
|Архангельская обл...|0.99795605518651|0.49936440677966104|0.49859164840684894|
+--------------------+----------------+-------------------+-------------------+
only showing top 1 row



Дисперсия по явке для каждого региона (по УИК)

In [99]:
df.groupBy('region').agg(F.stddev((sum([F.col(col)for col in counted_votes_cols])) / df.registered_voters).alias('std')).show()


+--------------------+-------------------+
|              region|                std|
+--------------------+-------------------+
|Республика Саха (...|0.11374120492897338|
|  Республика Хакасия|0.11719818576021894|
|Республика Ингушетия| 0.0697747654830594|
|Удмуртская Респуб...|0.09658085675037424|
|     Камчатский край|0.16861471602822437|
|Республика Башкор...|0.12284393684931046|
|Карачаево-Черкесс...|0.12319562690839514|
| Республика Дагестан|0.12935039014984046|
|Республика Татарс...|0.14150150293888705|
| Ставропольский край|0.13401966931050247|
|Республика Северн...|0.07930140237807017|
| Республика Марий Эл| 0.0818500080733599|
|Республика Адыгея...|0.16330124345063626|
|  Республика Бурятия|0.13306357237654534|
| Республика Калмыкия|0.15349925960543603|
|       Пермский край|0.10105338052282166|
|  Краснодарский край|0.13940623710253583|
|     Приморский край|0.16641672216989184|
|Кабардино-Балкарс...|0.03760708051294459|
|  Забайкальский край|0.11979231763260301|
+----------

Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [111]:
def get_candidate_table(cand, result_percent):
    temp = df.withColumn(f'{cand}_percent', F.ceil(100 * F.col(cand) / df.registered_voters))
    return temp.filter(F.col(f'{cand}_percent') ==  result_percent). \
        groupBy(F.col(f'{cand}_percent')) \
        .agg(F.count(F.col(f'{cand}_percent')).alias('uik_count')) \
        .sort(F.col('uik_count'), ascending=False)
        

candidates = df.columns[-8:]
for candidate in candidates:
   get_candidate_table(candidate, 20).show()


+---------------+---------+
|baburin_percent|uik_count|
+---------------+---------+
|             20|        2|
+---------------+---------+

+----------------+---------+
|grudinin_percent|uik_count|
+----------------+---------+
|              20|      362|
+----------------+---------+

+-------------------+---------+
|zhirinovsky_percent|uik_count|
+-------------------+---------+
|                 20|       57|
+-------------------+---------+

+-------------+---------+
|putin_percent|uik_count|
+-------------+---------+
|           20|       12|
+-------------+---------+

+---------------+---------+
|sobchak_percent|uik_count|
+---------------+---------+
|             20|        4|
+---------------+---------+

+----------------+---------+
|suraykin_percent|uik_count|
+----------------+---------+
+----------------+---------+

+-------------+---------+
|titov_percent|uik_count|
+-------------+---------+
|           20|        2|
+-------------+---------+

+-----------------+---------+
|y

In [110]:
def show_share_counts(cand, result):
    temp = df.withColumn(f'{cand}_share', F.ceil(100 * F.col(cand) / df.registered_voters))
    temp = temp.filter(F.col(f'{cand}_share') ==  result). \
        groupBy(F.col(f'{cand}_share')) \
        .agg(F.count(F.col(f'{cand}_share')).alias('count')) \
        .sort(F.col('count'), ascending=False) \
        .show()

cands_list = df.columns[-8:]
for c in cands_list:
    print(c)
    show_share_counts(c, 20)



baburin
+-------------+-----+
|baburin_share|count|
+-------------+-----+
|           20|    2|
+-------------+-----+

grudinin
+--------------+-----+
|grudinin_share|count|
+--------------+-----+
|            20|  362|
+--------------+-----+

zhirinovsky
+-----------------+-----+
|zhirinovsky_share|count|
+-----------------+-----+
|               20|   57|
+-----------------+-----+

putin
+-----------+-----+
|putin_share|count|
+-----------+-----+
|         20|   12|
+-----------+-----+

sobchak
+-------------+-----+
|sobchak_share|count|
+-------------+-----+
|           20|    4|
+-------------+-----+

suraykin
+--------------+-----+
|suraykin_share|count|
+--------------+-----+
+--------------+-----+

titov
+-----------+-----+
|titov_share|count|
+-----------+-----+
|         20|    2|
+-----------+-----+

yavlinsky
+---------------+-----+
|yavlinsky_share|count|
+---------------+-----+
+---------------+-----+

